<a href="https://colab.research.google.com/github/sanjulikasharma/ineuron/blob/main/video_generator_using_python_ipynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import files

# Upload your Kaggle API key (kaggle.json) file
uploaded = files.upload()
!pip install -q kaggle

# Move the uploaded Kaggle API key to the required directory
!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

# Download the dataset using Kaggle API
!kaggle datasets download -d rkuo2000/first-order-motion-model

Saving kaggle.json to kaggle.json
 99% 633M/636M [00:05<00:00, 152MB/s]
100% 636M/636M [00:05<00:00, 115MB/s]


In [ ]:
# Unzip the downloaded dataset
!unzip -q first-order-motion-model.zip -d /content/first-order-motion-model   # https://www.kaggle.com/datasets/rkuo2000/first-order-motion-model?select=vox-cpk.pth.tar

In [ ]:
!pip install ffmpeg
!pip install imageio-ffmpeg
!git clone https://github.com/rkuo2000/first-order-model

  Preparing metadata (setup.py) ... done
  Created wheel for ffmpeg: filename=ffmpeg-1.4-py3-none-any.whl size=6080 sha256=a1337365ad32c55b2de855988fa2ad5a07f4ca890c617b7838dd9218d3059aaa
  Stored in directory: /root/.cache/pip/wheels/8e/7a/69/cd6aeb83b126a7f04cbe7c9d929028dc52a6e7d525ff56003a
Successfully built ffmpeg
Cloning into 'first-order-model'...
remote: Enumerating objects: 339, done.
remote: Counting objects: 100% (33/33), done.
remote: Compressing objects: 100% (21/21), done.
remote: Total 339 (delta 15), reused 23 (delta 12), pack-reused 306
Receiving objects: 100% (339/339), 72.17 MiB | 24.68 MiB/s, done.
Resolving deltas: 100% (174/174), done.


In [ ]:
import os
os.chdir('first-order-model')
import imageio
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from skimage.transform import resize
from IPython.display import HTML
from demo import make_animation
from skimage import img_as_ubyte
from demo import load_checkpoints

In [ ]:
def display(source,driving,generated=None):
    fig=plt.figure(figsize=(8+4*(generated is not None),6))

    ims=[]
    for i in range(len(driving)):
        cols=[source]
        cols.append(driving[i])
        if generated is not None:
            cols.append(generated[i])
        im = plt.imshow(np.concatenate(cols, axis=1), animated=True)
        plt.axis('off')
        ims.append([im])
    ani=animation.ArtistAnimation(fig,ims,interval=50,repeat_delay=1000)

    plt.close()
    return ani

In [ ]:
generator, kp_detector = load_checkpoints(config_path='config/vox-256.yaml',
                            checkpoint_path='/content/first-order-motion-model/vox-cpk.pth.tar')

/usr/local/lib/python3.10/dist-packages/torch/functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3526.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


In [ ]:
source_image=imageio.v2.imread('/content/ariana.jpg')
driving_video=imageio.mimread('/content/Leonardo.mp4')

#resize
source_image=resize(source_image,(256,256))[...,:3]
driving_video=[resize(frame, (256, 256))[..., :3] for frame in driving_video]

predictions=make_animation(source_image,driving_video,generator,kp_detector, relative=True )

imageio.mimsave('/content/generated_video.mp4', [img_as_ubyte(frame) for frame in predictions]) # generated video saving loaction

  0%|          | 0/444 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/torch/nn/functional.py:4296: UserWarning: Default grid_sample and affine_grid behavior has changed to align_corners=False since 1.3.0. Please specify align_corners=True if the old behavior is desired. See the documentation of grid_sample for details.
  warnings.warn(


In [ ]:
# merge source_image, driving_video, animation into html video
HTML(display(source_image, driving_video, predictions).to_html5_video())